<hr>
<center><h1>El-Gamal : Elliptic Curve Encryption Algorithm</h1></center>
<hr>

## Curve P-256

In [79]:
# a large prime number p
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
# the finite field of Z/pZ
K = GF(p)
# equation parameters a and b such as : y^2 = x^3 + ax + b
a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
# Define the EC 
E = EllipticCurve(K, (a, b))
# the generator of the EC
G = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)
n =  0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551

## 1. KeyGen (Alice)

In [80]:
# Alice choses a random between 1 and o(G), sk as private key
# s = int(K.random_element(1, n))  # randomized for high probability
s = 0xffff
P = s*G

key_pair = (s, P)

## 2. Encryption (Bob)

<h3>Koblitz Method to transform a message  to a point on an elliptic curve for ElGamal</h3>
<ol>
    <li>Choose an encoding parameter k (e.g., 30, 50).</li>
    <li> Represent your message m as an integer.</li>
    <li> For j = 1 to k: </li>
    <ul>
        <li>Set x = m * k + j (mod p).</li>
        <li>Compute f(x) = x³ + a*x + b (mod p).</li>
        <li>Check if f(x) is a quadratic residue (i.e., has a square root) using Euler’s criterion.</li>
        <li>If yes, compute y = sqrt(f(x)). The point Pm = (x, y) is your message point.</li>    
    </ul>
    <li>If no j works after k tries, the algorithm fails (you increment m or change k).</li>

</ol>

In [81]:
# transorm the message to a point M over EC
message = "Hello Alice!"
message_bytes = message.encode('utf-8')

# Add null terminator and convert to integer
message_bytes = message_bytes + b'\x00'  
m = int.from_bytes(message_bytes, byteorder='big')

# Koblitz encoding
k = 100
M = None
for j in range(1, k):
    x_int = (m * k + j) % p
    x = K(x_int)
    fx = (x^3 + a*x + b) % p
    if fx.is_square():
        y = fx.sqrt()
        M = E(x, y)
        print(f"Encoded with j = {j}")
        break

# choose a random t to compute C1 and C2
t = int(K.random_element())
C1 = t * G
C2 = t * P + M

print(f"M = {M}\nC1 = {C1}\nC2 = {C2}")
ciphertext = (C1, C2)

Encoded with j = 1
M = (573581676307299953881327371084801 : 6966788150889191895609467632688083682831979045762676997523301920843479368695 : 1)
C1 = (76443370335214745592625834114865275169682946663702920882499206973401265015003 : 5111275973062424589481089563977738942882921675146311807494902421146443222353 : 1)
C2 = (1614220883196795146531383325556028361389337815560917443125286250109915285800 : 113872824766505230059446979422027803756419966724215411427729899084370123836676 : 1)


## 3. Decryption (Alice)

In [82]:
# to decrypt alice must compute : decrypted_M = C2 - s*C1
decrypted_M = ciphertext[1] - s * ciphertext[0]

# recover integer from point 
x_coord = int(decrypted_M[0])
recovered_m_int = int((x_coord - (x_coord % k)) // k)  

# convert back to bytes
byte_l = (recovered_m_int.bit_length() + 7) // 8
recovered_bytes = recovered_m_int.to_bytes(byte_l, 'big')
recovered_bytes = recovered_bytes.split(b'\x00')[0]
original_message = recovered_bytes.decode('utf-8')

print(f"Decrypted: {original_message}")

Decrypted: Hello Alice!


<hr>

## Attacker : can brute forcing to find secret key s

In [83]:
c = 100000  # this value is grater than d
for i in range(c + 1):
    Q = i * G
    print(f"Checking i={i} | Q={Q}", end="\r", flush=True)
    if P == Q :
        sk = i
        print(f"\nFound private key: {i}")
        break

Checking i=65535 | Q=(109588801698256994951990571109052358531364605021783910115998727948928103999695 : 82091379372138771455752671260931004134817620445688054187650988778738847793385 : 1))
Found private key: 65535
